# Initialisation and Dependencies

In [1]:
import sys
lib_path = '/home/jovyan/libs'
sys.path.insert(0, lib_path)

In [2]:
%reload_ext autoreload
%autoreload 2

import gc, math, traceback, datetime

import numpy as np

import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from datasets import load_from_disk

import whisper
from whisper.tokenizer import get_tokenizer

from utils import audio, gradient, gpu
from utils.attacks import PrepareFront, PrepareAtPosition

/usr/local/lib/python3.8/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.8/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


# GPU RAM Tracking

In [3]:
device = gpu.get_device()

Device: cuda


# Load Model

In [4]:
try:
    del model
    print("Model deleted!")
except:
    pass

In [5]:
SMALL_MODEL = "small.en"
TINY_MODEL = "tiny.en"

small_model = whisper.load_model(SMALL_MODEL).to(device)
small_model.eval()

tiny_model = whisper.load_model(TINY_MODEL).to(device)
tiny_model.eval()

target_id, sot_ids = gradient._get_ids(small_model)

/home/jovyan/libs/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


# Load Data

In [6]:
tedlium_path = "../tedlium"
train_path, validation_path, test_path = f"{tedlium_path}/train_idx.hf", f"{tedlium_path}/validation_idx.hf", f"{tedlium_path}/test.hf"

In [7]:
TRAIN_SELECT = 250
VALID_SELECT = 100
TEST_SELECT = 150

SEED = 1

tedlium_train = load_from_disk(train_path).with_format("torch").shuffle(seed=SEED).select(range(TRAIN_SELECT))
tedlium_validation = load_from_disk(validation_path).with_format("torch").shuffle(seed=SEED).select(range(VALID_SELECT))
tedlium_test = load_from_disk(test_path).with_format("torch").shuffle(seed=SEED).select(range(TEST_SELECT))

Loading dataset from disk:   0%|          | 0/109 [00:00<?, ?it/s]

In [8]:
# def collate(ls):
#     pad_to = max(list(map(lambda x: x["audio"].shape[0], ls)))
#     return torch.cat(list(map(lambda x: F.pad(x["audio"], (0, pad_to - x["audio"].shape[0])).unsqueeze(0).to(torch.bfloat16), ls)), dim=0)

def collate_idx(ls):
    return ls[0]["audio"].unsqueeze(0), ls[0]["idx"].item()

TRAIN_BATCH_SIZE = 1 # highly recommended to be 1
VALID_BATCH_SIZE = 1

train_dataset = DataLoader(tedlium_train, batch_size=TRAIN_BATCH_SIZE, collate_fn=collate_idx)
validation_dataset = DataLoader(tedlium_validation, batch_size=VALID_BATCH_SIZE, collate_fn=collate_idx)
test_dataset = DataLoader(tedlium_test)

# Training Loop

In [9]:
gpu.cleanup()

In [10]:
LR = 1e-3
PATIENCE = 7
MIN_LIMIT = None
ITER_LIMIT = None
CLAMP_EP = 0.005
SNIPPET_SIZE = (1, 10_240)
POSITION = 0
PREPARE_METHOD = PrepareAtPosition(SNIPPET_SIZE, POSITION)
DELTA = 0.01

CLAMP_EPS = (0.00125, 0.0025, 0.005)

writer = None

In [11]:
# tensorboard writer
# timestamp = datetime.datetime.now().strftime(f'%Y%m%d-%H%M%S_size_{SNIPPET_SIZE}_{PREPARE_METHOD.name}')
# writer = SummaryWriter(log_dir=f"../runs/size_tests/{timestamp}", max_queue=5)

In [12]:
best_snippet, snippets, _, _ = gradient.train(small_model, gradient.forward_multi,
                                              train_dataset, validation_dataset,
                                              PREPARE_METHOD,
                                              writer, lr=LR,
                                              train_success=None, valid_success=None,
                                              iter_limit=ITER_LIMIT, mins_limit=MIN_LIMIT, patience=PATIENCE, clamp_epsilon=CLAMP_EP, gap=DELTA)

/home/jovyan/libs/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


Prepare method: prepare_at_position
Snippet initialised to [1.0212062306891312e-06, 0.004999465774744749] of size (1, 10240)
Clamp: 0.005
Time Limit (Mins): None
Epochs Limit: None
Tracking training success: False
Tracking valid success: False


Training:   0% 0/1 [02:49<?, ?it/s, Iter 2, Training Batch 1/250]    

Trng Avg Loss: 7.408213442802429 | Valid Avg Loss: 7.685433864593506 | Patience: 7 | LR: [0.001] | Epoch Limit: None


Training:   0% 0/1 [05:13<?, ?it/s, Iter 3, Training Batch 1/250]    

...

Trng Avg Loss: 0.28705418469011784 | Valid Avg Loss: 0.6941412091255188 | Patience: 5 | LR: [1.5625e-05] | Epoch Limit: None


Training:   0% 0/1 [51:47<?, ?it/s, Iter 35, Training Batch 2/250]    

Trng Avg Loss: 0.28584949153661726 | Valid Avg Loss: 0.6941326260566711 | Patience: 4 | LR: [1.5625e-05] | Epoch Limit: None


Training:   0% 0/1 [52:15<?, ?it/s, Iter 35, Training Batch 178/250]


Cleared buffer
Cleared loss


In [13]:
# audio.view_mel(best_snippet.detach().to("cpu").squeeze())

# Evaluation

In [14]:
gradient.evaluate(small_model, best_snippet, PREPARE_METHOD, test_dataset, CLAMP_EP, POSITION) # commented to prevent the runtime from autorunning and crashing the thing

Clamp: 0.005
Prepare Method: prepare_at_position
Snippet Size: (1, 10240)
Position: 0


Inference: 100%|██████████| 150/150 [01:03<00:00,  2.37it/s, Valid Examples: 120 | Empty Sequences: 27 | Total SL: 8283 | Non-empty ASL: 89.06451612903226 | Total Bleu Score: 41.109153747558594]



Total valid examples: 120
Success rate (Empty): 0.225
Success rate (ASL): 69.025 (attacked) out of 122.75833333333334 (original)
Average Bleu Score: 0.3425762951374054
Average WER: 0.6654056953407352


In [15]:
# random_snippet = (torch.rand(SNIPPET_SIZE) - 0.5) * (CLAMP_EP / 0.5)
# print(torch.max(random_snippet), torch.min(random_snippet))
# gradient.evaluate(model, random_snippet, PREPARE_METHOD, test_dataset, CLAMP_EP, POSITION) # commented to prevent the runtime from autorunning and crashing the thing

# Batch Evaluation

In [16]:
# for C in (0.005,):
#     print(C)
#     best_snippet, _, _, _ = gradient.train(model, gradient.forward_auto,
#                                               train_dataset, validation_dataset,
#                                               PREPARE_METHOD,
#                                               writer, lr=LR, 
#                                               train_success=None, valid_success=None,
#                                               iter_limit=ITER_LIMIT, mins_limit=MIN_LIMIT, patience=PATIENCE, clamp_epsilon=C, gap=DELTA)
#     gradient.evaluate(model, best_snippet, PREPARE_METHOD, test_dataset, C, POSITION) # commented to prevent the runtime from autorunning and crashing the thing
#     print("\n")

In [17]:
# LENGTHS = (1600, 3200, 4800, 6400, 6800, 7200, 8000, 10240, 16000)

# for L in LENGTHS:
#     print(L)
#     PREPARE_METHOD = PrepareAtPosition((1, L), 0)
#     best_snippet, _, _, _ = gradient.train(model, gradient.forward_auto,
#                                               train_dataset, validation_dataset,
#                                               PREPARE_METHOD,
#                                               writer, lr=LR, 
#                                               train_success=None, valid_success=None,
#                                               iter_limit=ITER_LIMIT, mins_limit=MIN_LIMIT, patience=PATIENCE, clamp_epsilon=CLAMP_EP, gap=DELTA)
#     gradient.evaluate(model, best_snippet, PREPARE_METHOD, test_dataset, CLAMP_EP, POSITION) # commented to prevent the runtime from autorunning and crashing the thing
#     print("\n")

# Save Tensors

In [18]:
snippets = torch.stack(list(map(lambda x: x.cpu(), snippets)) + [best_snippet.cpu()])
snippets.shape

torch.Size([36, 1, 10240])

In [19]:
# torch.save(snippets.squeeze(), "snippets.pt")

In [20]:
# torch.save(torch.stack(list(map(torch.tensor, train_success.values()))), "train_success.pt")
# torch.save(torch.tensor(list(train_success.keys())), "train_ids.pt")

In [21]:
# torch.save(torch.stack(list(map(torch.tensor, valid_success.values()))), "valid_success.pt")
# torch.save(torch.tensor(list(valid_success.keys())), "valid_ids.pt")

# Save and Hear Snippet

In [22]:
# def normalise(random_snippet, ep):
#     # we assume torch.rand inits to [0, 1)
#     res = random_snippet * ep * 2 - ep
#     print(f"Normalised, Min {torch.min(res)}, Max {torch.max(res)}")
#     return res

In [23]:
# Save snippet to wav file
# save_audio(snippet, f"./snippets/clamp_{CLAMP_EP}_{PREPARE_METHOD.name}_snippet_only.wav")

In [24]:
# save_audio(PREPARE_METHOD(snippet.to("cpu"), tedlium_test[2]["audio"].unsqueeze(0)), f"./snippets/clamp_{CLAMP_EP}_{PREPARE_METHOD.name}_combined.wav")